In [18]:
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import twitter_samples, stopwords
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize
from nltk import FreqDist, classify, NaiveBayesClassifier
import re, string, random
# get the training data
positive_tweets = twitter_samples.strings('positive_tweets.json')
negative_tweets = twitter_samples.strings('negative_tweets.json')
stop_words = stopwords.words('english')

In [19]:
def remove_noise(tweet_tokens):
    cleaned_tokens = []

    for token, tag in pos_tag(tweet_tokens):
        # filter links and references
        token = re.sub('(http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|'
                       '(?:%[0-9a-fA-F][0-9a-fA-F]))+)|(@[A-Za-z0-9_]+)|((//)?t\.co/.*)|(\'..?)', '', token)
        if tag.startswith("NN"):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'

        lemmatizer = WordNetLemmatizer()
        token = lemmatizer.lemmatize(token, pos)

        if len(token) > 0 \
                and token not in string.punctuation \
                and token.lower() not in stop_words:
            cleaned_tokens.append(token.lower())
    return cleaned_tokens

In [20]:
# get token as feature: token=True
def get_tweets_for_model(cleaned_tokens_list):
    for tweet_tokens in cleaned_tokens_list:
        yield dict([token, True] for token in tweet_tokens)

In [21]:
positive_tweet_tokens = twitter_samples.tokenized('positive_tweets.json')
negative_tweet_tokens = twitter_samples.tokenized('negative_tweets.json')

positive_cleaned_tokens_list = []
negative_cleaned_tokens_list = []

for tokens in positive_tweet_tokens:
    positive_cleaned_tokens_list.append(remove_noise(tokens, stop_words))
for tokens in negative_tweet_tokens:
    negative_cleaned_tokens_list.append(remove_noise(tokens, stop_words))


In [22]:
# prepare the training data
positive_tokens_for_model = get_tweets_for_model(positive_cleaned_tokens_list)
negative_tokens_for_model = get_tweets_for_model(negative_cleaned_tokens_list)

positive_dataset = [(tweet_dict, "Positive")
                     for tweet_dict in positive_tokens_for_model]
negative_dataset = [(tweet_dict, "Negative")
                     for tweet_dict in negative_tokens_for_model]
dataset = positive_dataset + negative_dataset
random.shuffle(dataset)

train_data = dataset[:7000]
test_data = dataset[7000:]

In [23]:
from nltk import classify, NaiveBayesClassifier
classifier = NaiveBayesClassifier.train(train_data)

print("Accuracy is:", classify.accuracy(classifier, test_data))

print(classifier.show_most_informative_features(10))

Accuracy is: 0.995
Most Informative Features
                      :) = True           Positi : Negati =    994.5 : 1.0
                     sad = True           Negati : Positi =     25.9 : 1.0
                    glad = True           Positi : Negati =     22.6 : 1.0
                     bam = True           Positi : Negati =     19.9 : 1.0
                 awesome = True           Positi : Negati =     19.9 : 1.0
                follower = True           Positi : Negati =     19.2 : 1.0
                     x15 = True           Negati : Positi =     17.5 : 1.0
                     via = True           Positi : Negati =     16.0 : 1.0
                   didnt = True           Negati : Positi =     12.9 : 1.0
                    sick = True           Negati : Positi =     11.3 : 1.0
None


In [24]:
# save the model
import pickle, os
path = os.path.join('twitter_sentiment_project', 'sentiment', 'model', 'twitter_classifier.pickle')
f = open(path, 'wb')
pickle.dump(classifier, f)
f.close()